In [10]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [11]:
words = open('names.txt', 'r').read().split()

In [12]:
chars = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [13]:
block_size = 3

X, Y = [], []

for w in words[:5]:
    print(w)
    context = [0]*block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [48]:
X.shape

torch.Size([32, 3])

In [24]:
C = torch.randn((27, 2))

In [25]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [26]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [30]:
h =torch.tanh(emb.view(-1, 6) @ W1 + b1)
h

tensor([[ 0.9949, -0.9999, -0.4877,  ...,  1.0000, -0.9976,  0.9987],
        [ 0.8863, -0.9988,  0.9999,  ...,  0.9970, -0.9969,  0.7162],
        [ 0.9923, -0.3969, -0.9746,  ...,  0.9997, -0.9991,  0.9999],
        ...,
        [-0.9863, -0.9998, -0.9978,  ...,  0.9963,  0.7503, -0.8840],
        [-0.9947, -0.1373,  0.9697,  ..., -0.6452, -0.5708, -0.8145],
        [-0.7221, -0.1082, -1.0000,  ...,  0.3283, -0.5509,  0.6131]])

In [31]:
h.shape

torch.Size([32, 100])

In [32]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [33]:
logits = h @ W2 + b2

In [34]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)

In [35]:
loss = - prob[torch.arange(32), Y].log().mean()

In [36]:
loss

tensor(15.2815)

In [39]:
loss = F.cross_entropy(logits, Y)
loss

tensor(15.2815)

All together with cross_entropy $-$ cause it's more efficient

In [51]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [52]:
for p in parameters:
    p.requires_grad = True

In [54]:
for _ in range(10):
    #forward
    emb = C[X]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y)
    #backward
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data -= 0.1 * p.grad

print(loss.item())

6.350668907165527


Actually, we overfitted

Let's do the same, but with whole input

In [49]:
block_size = 3

X, Y = [], []

for w in words:
    context = [0]*block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [50]:
X.shape, Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))